In [3]:
import pandas as pd
import re

from sklearn.decomposition import NMF
from sklearn.preprocessing import Normalizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

data = pd.read_pickle("../../petition/petition_total_data.pkl")

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40054 entries, 0 to 40053
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   code      40054 non-null  object        
 1   sdays     40054 non-null  datetime64[ns]
 2   edays     40054 non-null  datetime64[ns]
 3   title     40054 non-null  object        
 4   count     40054 non-null  int64         
 5   content   40054 non-null  object        
 6   category  40054 non-null  object        
 7   progress  40054 non-null  object        
 8   link      40054 non-null  object        
 9   person    40054 non-null  object        
dtypes: datetime64[ns](2), int64(1), object(7)
memory usage: 3.1+ MB


In [5]:
data['progress'] = [re.sub("\n",'',progress) for progress in data['progress']]

In [6]:
data = data.reset_index(drop=True)
data.shape

(40054, 10)

In [7]:
data['sdays'] = pd.to_datetime(data['sdays'])
data['edays'] = pd.to_datetime(data['edays'])

data['count'] = pd.to_numeric(data['count'])

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

data['content'] = [re.sub('|'.join(['\n','\t','\r']),'',''.join(content)).strip() for content in data['content']]

data['content'] = [emoji_pattern.sub(r'', content) for content in data['content']]

In [8]:
sample_5 = data['title'][5:10]

In [9]:
sample_5 = list(sample_5)

In [10]:
sample_5

['안녕하세요. 증인 윤지오입니다.',
 '“정부아이돌봄서비스 아이돌보미 영유아 폭행 강력 처벌 및 재발방지방안 수립을 부탁합니다.”(14개월 아기가 아이돌보미에게  폭행을 당했습니다.)',
 '연합뉴스에 국민혈세로 지급하는 연 300억원의 재정보조금 제도의 전면 폐지를 청원합니다.',
 '소방공무원을 국가직으로 전환해주세요',
 '버닝썬 VIP룸 6인을 수사해 주세요']

In [11]:
from konlpy.tag import Hannanum, Kkma, Komoran, Okt
import time
morphs_processors= [('Hannanum', Hannanum()), ('Kkma', Kkma()), ('Komoran', Komoran()), ('Okt', Okt())]

for name, morphs_processor in morphs_processors:
    strat_time = time.time()
    morphs = [morphs_processor.morphs(sentence) for sentence in sample_5]                                          
    elapsed_time = time.time() - strat_time
    print('morphs_processor name = %20s, %.5f secs' % (name, elapsed_time))

# doc_nouns_list = [' | '.join(okt.nouns(doc)) for doc in sample_5]

# print(doc_nouns_list)

morphs_processor name =             Hannanum, 3.82178 secs
morphs_processor name =                 Kkma, 19.37319 secs
morphs_processor name =              Komoran, 0.03092 secs
morphs_processor name =                  Okt, 3.67599 secs


In [12]:
class List(list): 
    def __str__(self): 
        return "[" + ", ".join(["%s" % x for x in self]) + "]"

In [13]:
from konlpy.tag import Hannanum, Kkma, Komoran, Okt
import time

komoran= Komoran()
okt= Okt()
hannanum= Hannanum()
kkma= Kkma()

for sentence in sample_5:
    print("komran")  # 개발자분이 지속적으로 업데이트해줌, 시간이 빠름.
    print(komoran.pos(sentence))
    print("hannaum")
    print(hannanum.pos(sentence))
    print("okt")  # 인터넷 용어를 다룰 때 유용 
    print(okt.pos(sentence))
    print("kkma")
    print(kkma.pos(sentence))   # 세종 품사 태그 기반 
    print("---"*30)

komran
[('안녕하세요', 'NNP'), ('.', 'SF'), ('증인', 'NNP'), ('윤지오', 'NNP'), ('이', 'VCP'), ('ㅂ니다', 'EF'), ('.', 'SF')]
hannaum
[('안녕', 'N'), ('하', 'X'), ('세', 'E'), ('요', 'J'), ('.', 'S'), ('증', 'N'), ('이', 'J'), ('ㄴ', 'E'), ('윤지오입니', 'N'), ('이', 'J'), ('다', 'E'), ('.', 'S')]
okt
[('안녕하세요', 'Adjective'), ('.', 'Punctuation'), ('증인', 'Noun'), ('윤지오', 'Noun'), ('입니다', 'Adjective'), ('.', 'Punctuation')]
kkma
[('안녕', 'NNG'), ('하', 'XSV'), ('세요', 'EFN'), ('.', 'SF'), ('증인', 'NNG'), ('윤지', 'NNG'), ('오', 'NNG'), ('이', 'VCP'), ('ㅂ니다', 'EFN'), ('.', 'SF')]
------------------------------------------------------------------------------------------
komran
[('“', 'SS'), ('정부', 'NNP'), ('아이돌', 'NNP'), ('봄', 'NNP'), ('서비스', 'NNP'), ('아이돌', 'NNP'), ('보미', 'NNP'), ('영유아', 'NNG'), ('폭행', 'NNG'), ('강력', 'XR'), ('처벌', 'NNP'), ('및', 'MAJ'), ('재발', 'NNG'), ('방지', 'NNG'), ('방안', 'NNG'), ('수립', 'NNG'), ('을', 'JKO'), ('부탁', 'NNG'), ('하', 'XSV'), ('ㅂ니다', 'EF'), ('.', 'SF'), ('”', 'SS'), ('(', 'SS'), ('14', 'SN'), ('개

In [14]:
data['month'] = data['sdays'].dt.month

In [15]:
data_12 = data[data['month']==12]

data_12 = data_12.reset_index(drop=True)

In [16]:

doc_nouns_list = [' '.join(komoran.nouns(doc)) for doc in data_12['content']]

print(doc_nouns_list)

['11월 4일 어린이집 아파트 단지 내 어린이집 동갑내기 남자 아이 친구 앞 바지 항문 성기 손가락 아동 성폭력 피해 세 딸아이 아버지 아동 청소년의 성 보호 법제 조 정의 아동 청소년 이란 세 미만 자 말 세 도달 연도 1월 1일 자 제외 아동 청소년 대상 성범죄 다음 목 하나에 해당 죄 말 죄 딸 세 법 정의 아동 청소년 조 아동 청소년 강간 강제 추행 등 폭행 협박 아동 청소년 강간 사람 무기 징역 년 이상 유기 징역 아동 청소년 폭행 협박 다음 호 하나에 해당 행위 자 년 이상 유기 징역 구강 항문 등 신체 성기 제외 내부 성기 행위 성기 항문 손가락 등 신체 성기 제외 일부 도구 행위 딸 어린이집 아파트 단지 자전거 보관소 반 남자 아이 아동 청소년 강간 강제 추행 등 가해 아이 어린이집 반 친구 앞 딸 바지 항문 손가락 항문 손가락 성기 이로 인해 살 딸 질 진물 이나 아이 입 똥 말 딸 성범죄 피해자 가해 아동 법 정의 아동 청소년 대상 성범죄 성범죄 자 아동복지법 조 금지 행위 다음 호 하나에 해당 행위 개정 아동 매매 행위 아동 음란 행위 매개 행위 아동 성적 수치심 성희롱 등 성적 학대 행위 아동 신체 손상 신체 건강 발달 신체적 학대 행위 아동복지법 아동 성적 학대 행위 신체적 학대 행위 살 아이 말 형법 조 형사 미성년자 세 하지 형법 형사 미성년 자라 벌 벌 것 뿐 벌 안 유죄 무죄 수 유죄 형법 상 처벌 대상 표현 형사 처벌 대상 처음 고소 접수 안 현실 사례 가정 절망감 아동 청소년의 성 보호 법제 조 피해 아동 청소년 등 조치 청구 검사 성범죄 피해 아동 청소년 지속 위해 배제 보호 필요 인정 경우 법원 호 보호관찰 호 호 조치 청구 수 특정 범죄자 보호관찰 전자 장치 부착 등 법률 조의 항 호 호 가해자 특정 지역 출입 금지 등 준수 사항 부과 경우 가해자 대한 보호관찰 등 법률 보호관찰 피해 아동 청소년 주거 등 가해자 분리 퇴거 조치 피해 아동 청소년 주거 학교 등 미터 이내 가해자 가해자 대리인 접근 금지 조치 딸 어린이집

In [23]:

tfidf_vectorizer = TfidfVectorizer(min_df=2)  # 한 번만 나타나는 단어들은 무시
tfidf_matrix = tfidf_vectorizer.fit_transform(doc_nouns_list)


vector_array = tfidf_matrix.toarray()
nmf = NMF(n_components = 40, random_state = 0)  # 군집하
nmf.fit(vector_array)
features = nmf.transform(vector_array)

normalizer = Normalizer()
norm_features=normalizer.fit_transform(features)

print(norm_features[0:2])

df_features = pd.DataFrame(norm_features,index=data_12['title'].tolist())


[[0.00000000e+00 0.00000000e+00 3.24572155e-02 0.00000000e+00
  0.00000000e+00 1.25405330e-02 0.00000000e+00 1.23046185e-02
  2.99509735e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 2.18165289e-02 3.45324824e-02
  6.40019656e-02 1.08718067e-03 1.23602458e-02 8.36640810e-03
  0.00000000e+00 4.11131479e-03 3.91989604e-02 0.00000000e+00
  3.87116185e-03 0.00000000e+00 1.61269004e-03 4.18535166e-03
  0.00000000e+00 0.00000000e+00 9.48135149e-01 4.60663043e-04
  0.00000000e+00 0.00000000e+00 3.47727564e-02 1.41762009e-02
  0.00000000e+00 0.00000000e+00 3.13683257e-02 0.00000000e+00]
 [2.69446964e-01 4.64125528e-01 0.00000000e+00 1.50884366e-01
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 9.65911218e-03 0.00000000e+00
  3.70769191e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  3.70208833e-02 1.22608033e-02 4.27266079e-01 2.85177771e-01
  8.018

In [24]:
df_features.index

Index(['아동간 성폭력사고 시 강제력을 가진 제도를 마련해주시기 바랍니다.',
       '한국기독교총연합회 사단법인 해산과  전** 대표회장 구속을 촉구 합니다',
       '코로나전쟁에 왜 자영업자만 일방적 총알받이가 되나요? 대출원리금 임대료 같이 멈춰야 합니다.',
       '잔인하고도 무서운 학교폭력으로 우리아들의 인생이 망가졌습니다.', '불법도박의 피해가 점차 심각해지고 있습니다 도와주세요!!',
       '국회의원 세비 인상 반대합니다', '살인마들 다단계코인 사기범', '어린이 보육시설 cctv 상시 공개 의무화',
       '**고등학교의 횡포를 멈춰주세요!!', '인공관절 수술후 식물인간으로계시다가 돌아가셨습니다',
       ...
       '서울마포 3700세대 아파트에서 발암물질 페놀온수가 나옵니다', '전광훈을 무죄판결한 허선아 판사 탄핵 청원합니다.',
       '인천 서구 국공립 어린이집 장애 아동 집단 학대 사건', 'ai 조류독감 무자비한 살처분 정책을 막아주세요',
       '허선아 재판부 탄핵을 촉구합니다.', '****의 신용점수제는 대국민 사기극!',
       '서울 마을버스 업체 지원 정부에서 해결해주세요', '과거사위원회를 상설기구로 설치하여 주십시오.',
       '정부와 보건복지부에 요청 합니다 의대생 국시 절대 반대 합니다',
       '내년 의사 국가고시를 1월에 추가로 실시하면서 정부를 믿고 올해 응시한 423명을 배신해 놓고, 그들을 위해서는 아무런 말도 하지 않았습니다. 여기에 대한 대책을 요구합니다.'],
      dtype='object', length=1348)

In [25]:
article = df_features.loc['코로나전쟁에 왜 자영업자만 일방적 총알받이가 되나요? 대출원리금 임대료 같이 멈춰야 합니다.']
similarities = df_features.dot(article)
top = similarities.nlargest(5)

for i in range(1, len(top)):
    print("Title: "+top.index[i]+'\n Similarities'+str(top[i]))

Title: 자영업자의 재산권침해에 대한 피해보상을 촉구합니다.
 Similarities0.9940695541034386
Title: 긴급,재난시 발효하는 사업장의 영업정지를 임대정지로 바꿔 주세요
 Similarities0.9926898218705152
Title: 12월8일부로 명령된 코로나 집합금지 대상 업종 임대료 의무 삭감 요청
 Similarities0.9920831461179165
Title: 답답해서 다시올립니다 !!  코로나전쟁에 일방적으로 희생되는 총알받이 자영업자는 이 나라의 국민이 맞습니까?
 Similarities0.9910493527208297


In [26]:
article = df_features.loc['아동간 성폭력사고 시 강제력을 가진 제도를 마련해주시기 바랍니다.']
similarities = df_features.dot(article)
top = similarities.nlargest()

for i in range(1, len(top)):
    print("Title: "+top.index[i]+'\n Similarities'+str(top[i]))

Title: 아동 살인한 범죄 신상공개 해주세요!! 최근에도  아동을  그것도  친자식을 끔찍하게  살해한  사건이  있습니다.
 Similarities0.9515337493118482
Title: 아동보호전담요원의 어려움과 힘듦을 들어주세요!! 아동을 보호하고 학대예방을 하기 위해 열심히 뛸수 있도록 힘을 실어 주세요!!
 Similarities0.9474183090464655
Title: 아동학대전담공무원의 목소리를 들어주세요
 Similarities0.946309904537855
Title: 어린이집 학대 방지를 위해 CCTV의 열람 상시화를 요구합니다.
 Similarities0.9449221791184209


In [27]:
article = df_features.loc['정부와 보건복지부에 요청 합니다 의대생 국시 절대 반대 합니다']
similarities = df_features.dot(article)
top = similarities.nlargest()

for i in range(1, len(top)):
    print("Title: "+top.index[i]+'\n Similarities'+str(top[i]))

Title: ‘자의로’ 시험을 거부한 의대생의 구제를 반대합니다
 Similarities0.9792242925441598
Title: 형평성, 공정성, 윤리적인 면에서 벗어난 국시거부 의대생 재응시 절대 반대합니다.
 Similarities0.9665408851444357
Title: 의사국시 재시험 반대합니다.
 Similarities0.9659010087744154
Title: 정세균 국무총리 사퇴해주세요
 Similarities0.9579929069668364


In [22]:
#  Komoran이 okt보다 더 좋은 결과.